In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..", "tools")))
sys.path.append(os.path.abspath(os.path.join("..", "db")))
sys.path.append(os.path.abspath(os.path.join("..", "rag")))


In [2]:
from envvars import load_env_vars_from_directory
import pandas as pd
import numpy as np
import sqlite3
from sttcastdb import SttcastDB
import os
from openai import OpenAI
import json

In [3]:
load_env_vars_from_directory("../.env")
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
dbfile = "/home/jmrobles/Podcasts/Coffee Break/DB/coffeebreak.db"
db = SttcastDB(dbfile)

In [5]:
fromdate = "2015-01-01"
todate = "2026-01-01"

In [6]:
ints = db.get_pending_ints(fromdate=fromdate, todate=todate)

In [7]:
from tqdm import tqdm

In [8]:
def get_text_from_iv(iv):
    return (f"[Episodio {iv['epname']}] "
           f"[Fecha: {iv['epdate']}] "
           f"[Hablante: {iv['tag']}] "
           f"[Inicio: {iv['start']}s Fin: {iv['end']}s] "
           f"{iv['content']}")


In [9]:
for i in ints[:10]:
    print(get_text_from_iv(i))

In [10]:
model = "text-embedding-3-small"
batch_size = 100

In [ ]:
ptk = 0
ttk = 0
for i in tqdm(range(0, len(ints), batch_size), desc="Batches"):
    batch = ints[i:i+batch_size]
    ids = []
    texts = []
    for iv in batch:
        ids.append(iv['id'])
        txt = get_text_from_iv(iv)
        # Monta el texto con metadatos + contenido
        texts.append(txt)

    resp = openai.embeddings.create(
        model=model,
        input=texts
    )
    usage = resp.usage
    prompt_tokens = int(usage.prompt_tokens / batch_size)
    total_tokens = int(usage.total_tokens / batch_size)
    ptk += usage.prompt_tokens
    ttk += usage.total_tokens
  
    for _id, data in zip(ids, resp.data):
        emb = np.array(data.embedding, dtype=np.float32).tobytes()
        db.update_embedding(_id, emb, prompt_tokens, total_tokens)

db.commit()

print("Embeddings con metadatos actualizados.")
print(f"Consumo para {len(ints)} intervenciones: prompt_tokens = {ttk} - total_tokens = {ptk}")

Batches: 0it [00:00, ?it/s]

Embeddings con metadatos actualizados.
Consumo para 0 intervenciones: prompt_tokens = 0 - total_tokens = 0


In [12]:

# Convierte cada embedding desde BLOB a np.ndarray

ints = db.get_embedded_ints(fromdate=fromdate, todate=todate)

embeddings = [
    np.frombuffer(intv["embedding"], dtype=np.float32)
    for intv in ints
]

metadata = [
    {
        "id": intv["id"],
        "text": intv["content"],
        "epname": intv["epname"],
        "epdate": intv["epdate"],
        "tag": intv["tag"],
        "start": intv["start"],
        "end": intv["end"]
    }
    for intv in ints
]
print(f"len(embeddings) = {len(embeddings)} - len(metadata) = {len(metadata)}")

len(embeddings) = 201601 - len(metadata) = 201601


In [13]:
import faiss

In [14]:
FAISSFILE = '/home/jmrobles/Podcasts/Coffee Break/DB/int_index.faiss'
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)

vectors = np.stack(embeddings)
index.add(vectors)

faiss.write_index(index, FAISSFILE)

In [53]:
# query = "¿Qué se ha dicho sobre un templo de más de 10000 años de antigüedad que se ha descubierto en Turquía?"
# query = "¿Qué discrepancias han señalado los contertulios con los trabajos de Van Dokum?"
# query = "¿Qué se ha hablado sobre un instrumento de navegación de la época antigua griega"
# query = "¿Que opinan los contertulios, 68 o 73?"
# query = "Entonces, según los contertulios, Einstein se equivoca ¿no?"
# query = "¿Cuáles son los argumentos de Nacho Trujillo para cuestionar el trabajo de Van Dokum? ¿Qué opina el resto de contertulios?"
# query = "¿Cuándo se habló y quién lo hizo sobre un artefacto griego que aparecía en una película sobre Arquímedes?
# query = "¿Desde cuándo son contertulias Sara Robisco y María Ribes"
# query = "¿Cuáles son los candidatos más sólidos como constituyentes de la materia oscura? ¿Hay algún contertulio que se decanta por alguno?"
# query = "¿En qué episodios se explica la supersimetría? ¿Quién lo hace?"
# query = "Por favor, señálame los episodios en los que se hayan dicho cosas más relevantes sobre Alpha Fold"
# query = "¿Qué recomendaciones se hicieron entre enero y marzo de 2020 ante la inminente pandemia?"
# query = "¿En qué episodios se ha explicado en modelo estándar de partículas"
# query = "¿En qué episodios se explica CRISPR"
# query = "¿Han explicado los contertulios cuál es la naturaleza de la energía oscura?"
# query = "¿Qué se ha dicho de la estrella de Tapi?"
# query = "Recuerdo que en algún episodio se discutió sobre inteligencias artificiales que utilizan el sueño para mejorar su comportamiento ¿puedes decirme cuál es y resumirme qué se dijo?"
# query = "¿En qué consiste la mala praxis que se le achaca a un profesor que hinchaba sus méritos? ¿a quién se refieren?"
# query = "¿Qué se ha dicho sobre el proceso de pulido de las lentes de los telescopios? ¿Se han mencionado proveedores de herramientas?"
# query = "¿Se ha hablado algo sobre tecnología asociada a la producción de diamantes industriales?"
# query = "¿Cuáles son los argumentos para sostener la existencia del planeta 9? ¿Qué contertulios los exponen?"
# query = "¿Quién es Operador Nuclear? ¿De qué ha hablado?"
# query = "En alguno de los episodios oí que, para conseguir variedades de plantas, se utilizaba radioactividad ¿en qué episodio o episodios se habló de este tema? ¿Quién lo hizo?"
# query = "¿Estamos lejos, según los participantes en la tertulia, de que un ordenador cuántico rompa la codificación del bitcoin?, ¿qué haría falta?"
# query = "¿Les gusta a los contertulios el número diecisiete?"
# query = "¿Se han encontrado restos del meteorito que cayó en Siberia a principios del siglo XX?"
# query = "¿Qué se ha dicho de la empresa Clarivate?"
# query = "¿Por qué se duda del hallazgo de Fosfano en Venus y de que sea un biomarcador?"
# query = "¿Se ha hablado algo de los posibles recursos mineros de uranio en España? ¿y en el mundo?"
# query = "¿Qué posibilidades tiene el torio como combustible nuclear?"
# query = "¿Cómo funcionan las vacunas de ARN mensajero?"
# query = "La medición de ondas gravitacionales, ¿qué nos dice sobre la población de agujeros negros?"
# query = "¿Cuándo y de qué habló Abraham Loewe en el programa?"
# query = "En algún programa se comentó una forma extraña de tomar café, ¿en cuál fue? ¿qué más se dijo?"
# query = "¿Cuándo se descubrió que la principal vía de infección de la COVID era la aérea?"
# query = "¿Se han descubierto muchos planetas enanos?"
# query = "¿Por qué el oro es dorado? ¿en qué episodio se contó?"
# query = "¿Se ha hablado de dimetilsufuro"
# query = "¿Qué se ha dicho sobre el mundo hiceánico"
# query = "¿En qué episodios ha hablado David Kipping?"
query = "¿En qué episodios ha hablado Rafael Luque?"

In [54]:
qresp = openai.embeddings.create(
    model=model,
    input=[query]
)
qvec = np.array(qresp.data[0].embedding, dtype=np.float32).reshape(1, -1)
print(qresp.usage.total_tokens)

14


In [55]:
relevant_fragments = 100

In [56]:
faiss.normalize_L2(qvec)
D, I = index.search(qvec, k=relevant_fragments)
resultados = [metadata[i] for i in I[0]]
print(resultados)

[{'id': 55118, 'text': 'es un investigador joven que se llama Rafael Luque, que está haciendo el doctorado aquí con Enric Payet, que es el segundo autor. Enric lo hemos tenido un par de veces en el programa hablando de exoplanetas, uno de los expertos mundiales en temas de exoplanetas, y está metido en todas estas historias. Por lo que cuenta Rafael Luque, ellos tiraron de muchas observaciones de archivos que ya se tenían de esta estrella,', 'epname': 'ep226', 'epdate': datetime.date(2019, 8, 1), 'tag': 'Héctor Socas', 'start': 7716.45, 'end': 7746.15}, {'id': 123677, 'text': 'es un investigador joven que se llama Rafael Luque, que está haciendo el doctorado aquí con Enric Payet, que es el segundo autor. Enric lo hemos tenido un par de veces en el programa hablando de exoplanetas, uno de los expertos mundiales en temas de exoplanetas, y está metido en todas estas historias. Por lo que cuenta Rafael Luque, realmente ellos tiraron de muchas observaciones de archivos que ya se tenían de e

In [57]:
gpt_model="gpt-4o-mini"
contexto = "\n\n".join(f"{r['tag']} en {r['epname']}, [{r['epdate']}], a partir de {r['start']} :\n{r['text']}" for r in resultados)

prompt = f"""Eres un asistente experto en podcasts de divulgación científica. Usa el siguiente contexto para responder con precisión:

{contexto}

Por favor, cíñete lo más posible al contexto. Puedes añadir algo fuera de ese contexto, pero indicándolo.

Al final de la respuesta haz una lista de los fragmentos más relevantes (alrededor de 10) donde se habló incluyendo fecha, episodio,
hablante, momento a partir del que se habla y primeros 50 caracteres del contenido de la intervernción. Ponlo en formato
tabla de markdown

Pregunta: {query}
Respuesta:"""

respuesta = openai.chat.completions.create(
    model=gpt_model,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)
print("PREGUNTA")
print(f"{query}")
print("="*50)
print("="*50)
for i, resp in enumerate(respuesta.choices):
    print(f"RESPUESTA {i}")
    print(resp.message.content)
    print("="*50)
print(f"Número de prompt_tokens: {respuesta.usage.prompt_tokens} - Número total de tokens {respuesta.usage.total_tokens}")



PREGUNTA
¿En qué episodios ha hablado Rafael Luque?
RESPUESTA 0
Rafael Luque ha sido mencionado en varios episodios del podcast "Coffee Break: Señal y Ruido". A continuación, se detallan los episodios en los que se ha hablado de él, junto con el contexto relevante:

1. **Episodio 226 (2019-08-01)**: Héctor Socas menciona a Rafael Luque como un investigador joven que está haciendo su doctorado con Enric Payet, un experto en exoplanetas. Se habla sobre las observaciones de archivos que utilizaron para investigar una estrella.

2. **Episodio 227 (2019-08-08)**: Se repite la misma información sobre Rafael Luque y su trabajo con Enric Payet, enfatizando el uso de observaciones archivadas.

3. **Episodio 411 (2023-04-13)**: Se menciona un incidente donde un estudiante de Rafael Luque se apropió de datos de otros investigadores, lo que generó controversia.

4. **Episodio 411 (2023-04-13)**: Se discute una sentencia condenatoria relacionada con Rafael Luque y su situación legal en la Universid

In [ ]:
from sttcast_rag_service import RelSearchRequest
import requests
from urllib.parse import urljoin

In [21]:
for r in resultados:
    r['epdate'] = str(r['epdate'])
    
relquery = RelSearchRequest(
    query=query,
    embeddings=resultados
).model_dump()
relquery

{'query': '¿Se han descubierto muchos planetas enanos?',
 'embeddings': [{'tag': 'Héctor Socas',
   'epname': 'ep323',
   'epdate': '2021-07-08',
   'start': 2597.21,
   'end': 2622.33,
   'text': 'con la idea de lo que son planetas enanos. Son dos cosas diferentes, ¿vale? Una cosa es planeta menor, minor planet, y otra cosa es planeta enano, dwarf planet. Planeta menor no está muy bien definido lo que es. Es una nomenclatura que se usa de forma muy genérica y muy ambigua para decir cualquier cosa en el sistema solar, que pueden ser asteroides grandes.'},
  {'tag': 'Carlos Westendorp',
   'epname': 'ep221',
   'epdate': '2019-06-27',
   'start': 1420.21,
   'end': 1439.9,
   'text': 'Muy bien, me parece muy interesante. Ya se han descubierto 4.000 planetas alrededor de estrellas enanas. Esta es una estrella especialmente pequeña y lo interesante es lo que dice al final, que es la primera estrella con planetas que se detecta en la eclíptica.'},
  {'tag': 'Alberto Aparici',
   'epname': 

In [22]:
rag_service_url = "http://localhost:5500"
relsearch_path = "/relsearch"
relsearch_url = urljoin(rag_service_url, relsearch_path)
relsearch_url


'http://localhost:5500/relsearch'

In [76]:
response = requests.post(relsearch_url, json=relquery)
print(response.ok)

True


In [77]:
print(response)

<Response [200]>


In [78]:
json.loads(response.text)

{'tokens_prompt': 11838,
 'tokens_completion': 502,
 'tokens_total': 12340,
 'estimated_cost_usd': 0.002077,
 'search': {'es': 'El descubrimiento de que la principal vía de infección de la COVID-19 era la aérea se consolidó a medida que avanzaba la pandemia. Desde el inicio, se observó que el virus se propagaba principalmente a través de gotículas respiratorias y aerosoles. En episodios como el ep344, Héctor Socas menciona cómo se ha estudiado la transmisión del SARS-CoV-2 a través del aire, destacando la dificultad de medir la cantidad de virus en diferentes tamaños de gotas. Además, en el ep339, Francis Villatoro refuerza esta idea al explicar que el contagio ocurre principalmente por vía nasofaringia, lo que implica que la transmisión aérea es clave. A lo largo de la pandemia, se han realizado numerosos estudios que han confirmado esta forma de transmisión, lo que ha llevado a recomendaciones sobre el uso de mascarillas y la ventilación de espacios cerrados para reducir el riesgo de